# Нормализация сцен Ландсат

Даны разновременные сцены Ландсат, покрывающие одну и ту же территорию. Яркости пикселей разных сцен изначально несравнимы между собой вследствие различий в условиях съемки. Требуется промасштабировать яркости пикселей этих сцен таким образом, чтобы появилась возможность сравнивать их между собой и отслеживать динамику изменений яркостей. Эта процедура называется относительной нормализацией.

Нормализация, применяемая в данном документе, следует методике из статьи Du Y., Teillet P. M., Cihlar J. Radiometric normalization of multitemporal high-resolution satellite images with quality control for land cover change detection //Remote sensing of Environment. – 2002. – Т. 82. – №. 1. – С. 123-134.

Основные положения статьи:

1. Если выполняются определенные условия, в частности, если нелинейные эффекты от снимка к снимку незначительны по сравнению с линейными, и искажения равномерно распределены в пространстве, то можно производить линейную нормализацию. (Более точная формулировка условия дана в следующем разделе.)
2. Для линейной нормализации следует выбрать псевдоинвариантные объекты (PIF) и линейно преобразовать яркости пикселей таким образом, чтобы статистики яркостей (среднее и дисперсия), вычисленные по выбранным PIF совпадали у различных сцен.
3. PIF можно выбирать вручную, но это долго и не объективно. В статье предлагается процедура автоматического выбора PIF. Поскольку эта часть статьи наиболее интересна для нас и на ней основана вся методика, то она рассматривается ниже в отдельном разделе.

## Методика выбора PIF: теория

### Ограничения метода

Предположим, что выполняются следующие условия:

1. Отражающая способность объектов, выбранных в качестве PIF и используемых в процедуре нормализации, является постоянной в течении анализируемого периода времени.
2. Все изменения яркостей пикселей (DN), которые фиксируются на снимках для выбранных PIF, не являются реальным изменением объектов на местности.
3. Все изменения DN в течении анализируемого периода времени линейны и пространственно однородны для всех PIF, учавствующих в процедуре нормализации.
4. Все эффекты, приводящие к изменениям DN, независимы друг от друга и их результирующая величина представляет собой линейную комбинацию отдельных эффектов.
5. Реальные изменения объектов сцены не являются массовыми.

В этом случае процедура относительной нормализации сцен может быть реализована на базе линейного преобразования. 

**Замечание.** В рассматриваемой статье предполагается, что это DN (при отосительной нормализации не обязательно переходить от DN к ToAR, поскольку этот переход линеен и, следовательно, будет учтен при нормализации), но мы будем работать не с DN, а с ToAR и данными, прошедшими топографическую коррекцию. Этот выбоор обусловлен тем, что нам необходимо произвести маскирование облачности (оно оперирует ToAR), а также тем, что в случае горных условий Дальнего Востока очень желательно произвести топографическую коррекцию, поскольку нужно нивелировать влияние склонов и положения Солнца (без этого возникнет явное противоречие с условием пространственной однородности).

### Нормализация

Пусть Q(i) -- яркость пикселя PIF номер i. Линейная нормализация подразумевает преобразование изображений 1, 2, ..., j, ..., m по следующей формуле:
$$
N_j(i) = Q_j(i)\times \alpha_i + \beta_i = Q_j(i)\times \frac{\sigma_{ref}^{(Q)}}{\sigma_j^{(Q)}} +  \overline{Q}_{ref} \frac{\sigma_{ref}^{(Q)}}{\sigma_j^{(Q)}}\times \overline{Q}_j
$$
где $N_j(i)$ - нормированное значение i-го PIF в j-м изображении, $\overline{Q}_{ref}$, $\sigma_{ref}^{(Q)}$ - желаемые статистики средних яркостей и дисперсии, рассчитанных по всем PIF (они могут быть произвольными константами или же вычислены на базе произвольного изображения), а $\overline{Q}_j$ и $\sigma_j^{(Q)}$ - средняя яркость и дисперсия для PIF в изображении j.

### Стратегии выбора PIF

На практике PIF выбираются или руками (что нам не интересно), или автоматически. Будем рассматривать второй способ. 

Поскольку выше были сформированы условия линейности, то при их выполнении мы можем расчитывать на то, что PIFы между собой связаны линейным законом. Это означает, что если мы возьмем два изображения и преобразуем их при помощи метода главных компонент, то основная информация будет сконцентрирована вдоль одной оси (первой главной компоненты), вдоль второй оси будут собраны шумы и изменения. Это обосновывает (в статье рассказывается несколько подробнее) следующую процедуру поиска PIF:

1. Очистка данных от заведомо нестабильных пикселей: (а) применить порог для удаления облачных пикселей; (б) удалить пиксели, соответствующие водным объектам (они быстро меняются); (в) удалить все пиксели, для которых разность между первым и вторым изображением выше некоторого порога (они сигнализируют о потенциальном изменении, опасно выбирать их в качестве PIF).
2. Произвести преобразование оставшихся пикселей выбранной пары изображений методом главных компонент.
3. Все пиксели, лежащие вдоль главной оси на расстоянии не большем $l$ единиц (т.е. $-l < minor < l$, где minor - вторая главная компонента) маркируются как потенциальные PIF.
4. Если выбранные PIF удовлетворяют критериям качества (см. ниже), то процедура поиска останавливается. Если нет, то уменьшается величина $l$ (а также, возможно, ужесточаются пороговые значения) и производится повторный поиск PIF при новых параметрах.
5. Вычисляются среднее и дисперсия яркостей выбранных PIF (величины $\overline{Q}_j$ и $\sigma_j^{(Q)}$), которые в дальнейшем используются при нормализации.

**Замечание.** В статье рассматриваются, по всей видимости, только одноканальные изображения или же многоканальные, но при этом каждый канал анализируется отдельно от остальных. В случае многоканальных изображений, вероятно, такой подход приведет к некторому искажению в отношении каналов: $B_k/B_m \approx N_k/N_m (?)$ (Другими словами, после нормировки спектральные вектора подвергнутся некоторому повороту; небольшой поворот не страшен, он может объясняться шумами, но нет гарантии, что этот поворот не окажется больше, чем следует). Т.е. по-хорошему, нужно доработать процедуру, предоженную в статье, для того, чтобы можно было использовать многоканальные изображения без искажений спектральных векторов объектов. (Ну или убедиться, что данная процедура не приводит к такому искажению). На текущем этапе оставим все как есть.

### Оценка качества PIF

Для оценки качества PIF авторы предлагают пройти несколько стадий/способов оценки:

1. Посчитать коэффициент корреляции между выбранными PIF. Авторы предлагают считать набор не подходящим, если коэффициент не превышает значение 0.9. Этот метод должен неплохо срабатывать на этапе выбора порогов разности изображений и/или порогов облачности и водных объектов.
2. После применения метода главных компонент оценкой качества выбранных PIF может служить величина наклона главной оси: чем ближе она к 45 градусам, тем лучше. Итеративным подбором величины $l$ можно добиться постепенного приближения наклона главной оси к желаемому.
3. Этот способ применим, если производится нормализация не одной пары, а нескольких пар снимков. Например, имеются три снимка A, B и C. Тогда можно применить метод главных компонент к парам AB, AC, BC. При этом не исключено, что параметры нормализации, полученные для разных пар могут быть разными (зная парамаметры нормализации для пар AB и BC можно получить параметры для AC, которые могут противоречить вычисленным напрямую для AC значениям). Поэтому еще одним критерием качества может служить близость параметров нормализации, вычисленных разными способами.

## Нормализация: практика

### Процедура выбора PIF

Создадим функцию, которая будет брать на вход названия двух растров и искать на них PIF, результат . Функция вернет словарь с описанием процесса работы:
 
 * b: коффициент регрессии между PIF (тангенс угла наклона линии регрессии), чем ближе к единице, тем лучше;
 * N: число PIF;
 * iterations: число выполненных итераций;
 * good_quality: True, если процедура сошлась, и False в противном случае.

**TODO:** Сейчас реализация просто даляет маску (растр MASK), если она была. По-хорошему, нужно сделать проверку того, что растр маски присутствует и восстанавливать его после работы функции.

In [ ]:
def find_pif(rast1, rast2, result, grass):
    
    ###### Params for flow control and quality assesment #########
    diff_eps = 0.2  # Init threshold for the difference between input rasters
    diff_eps_rate = 0.9  # Rate for decrease diff_eps param during every iteration.
    
    l_eps = 0.01  # Init threshold for l (l < minor < l)
    l_eps_rate = 0.9  # Rate for decrease l_eps param during every iteration.
    
    # Tolerance for regression line coef: abs(b - 1) < regression_eps
    # where PIF_rast1 = a + b*PIF_rast2; b(very_good_quality) == 1
    regression_eps = 0.001 
    
    # If count (unmasked pixels) < then min_pixels, exit. 
    # good_quality := False
    min_pixels = 1000   
    
    max_iteration = 25
    
    ###### End of params for flow control and quality assesment #########
    
    
    good_quality = False  # Are the PIFs good?
    
    current_iteration = 0
    while (not good_quality) and (current_iteration < max_iteration):
        current_iteration += 1
        try:
            grass.run_command('r.mask', flags='r')
        except:
            pass

        # Step 1
        try:
            tmp_diff_name = 'tmp' + uuid.uuid4().hex
            expr = '%s = abs(%s - %s)' % (tmp_diff_name, rast1, rast2)
            grass.run_command('r.mapcalc', expression=expr)
            grass.run_command('r.mapcalc', 
                              expression="MASK=if(%s < %s, 1, 0)" % (tmp_diff_name, diff_eps))
        finally:
            grass.run_command('g.remove', type='rast', name=tmp_diff_name, flags='f')
    
        try:
            # Step 2
            tmp_pca_name = 'tmp' + uuid.uuid4().hex
            grass.run_command('i.pca',
                              input=','.join([rast1, rast2]),
                              output=tmp_pca_name,
                              rescale='0,0'
                             )
            # Step 3
            grass.run_command('r.mask', flags='r')
            # PCA minor is stored in 'xxxxx.2' raster (we have 2 rasters only)
            expr = '%s = if(abs(%s) > %s, 0, 1)' % ('MASK', tmp_pca_name+'.2', l_eps)
            grass.run_command('r.mapcalc', expression=expr)
        finally:
            grass.run_command('g.remove', type='rast', pattern=tmp_pca_name+'*', flags='f')
    
        # Step 4
        # regression line quality:
        coefs = grass.read_command('r.regression.line',
                          mapx=rast1, mapy=rast2, flags='g'
                        )
        for line in coefs.split():
            if 'b=' == line[:2]:
                b = float(line[2:])
            elif 'N=' == line[:2]:
                N = int(line[2:])

        if N < min_pixels:
              # Next iterations don't increase count of umnasked pixels
            return {'good_quality': False, 
                    'iterations': current_iteration,
                    'N': N
                   }
        
        if abs(b - 1) < regression_eps:
            # PIF are found
            good_quality = True
            grass.run_command('g.copy', rast=','.join(['MASK', result]), overwrite=True)
            
        diff_eps *= diff_eps_rate
        l_eps *= l_eps_rate
    
    grass.run_command('r.mask', flags='r')
    
    result_info = dict(
        good_quality=good_quality,
        N=N,
        b=b,
        iterations=current_iteration
    )
                              
    return result_info



In [ ]:
import os
import uuid

import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)


from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='landsat', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

Пример запуска:

In [ ]:
rast1 = 'clean.topo.toar_LC81120272014106LGN00_B2'
rast2 = 'clean.topo.toar_LC81120272014090LGN00_B2'
grs.grass.run_command('g.region', rast=rast1, res='120')

find_pif(rast1, rast2, 'tmp_PIF', grs.grass)